::::
:::{thebe-button}
:::
::::

# Find tracks

Compare experimental bubble histories to bubble history correlations. Bubble detection and linking performed by Trackpy, an implementation of the Crocker-Grier algorithm {cite}`allanTrackpy2018,crockerMethodsDigitalVideo1996`.


In [ ]:
from boilercv_docs.nbs import init

paths = init()

from numpy import diff, linalg
from pandas import DataFrame, read_hdf

from boilercv_docs.nbs import HIDE, display_dataframe_with_math, nowarn, style_df
from boilercv_pipeline.experiments.e230920_subcool import (
    GBC,
    M_TO_MM,
    OBJECTS,
    THERMAL_DATA,
    Col,
    transform_cols,
)
from boilercv_pipeline.sets import get_dataset

with nowarn(capture=True):
    from trackpy import link, quiet

quiet()

TIME = "2023-09-20T17:14:18"
"""Timestamp of the trial to be analyzed."""

# Track tuning
SEARCH_RANGE = 30
"""Pixel range to search for the next bubble."""
MEMORY = 5
"""Frames to remember a bubble."""

# Physical parameters
PX_PER_M = 20997.3753

HIDE

In [ ]:
path_time = TIME.replace(":", "-")
video = get_dataset(TIME.replace(":", "-"))["video"]
frametime = diff(video.time.values).mean()
objects: DataFrame = read_hdf((OBJECTS / f"objects_{path_time}").with_suffix(".h5"))  # type: ignore
subcooling = read_hdf(THERMAL_DATA).subcool[TIME]

HIDE

## Find bubbles in each frame and link them

Detect individual bubbles in each frame, and then link detections across frames by application of the Crocker-Grier tracking algorithm, which takes into account centroid proximity and expected positions {cite}`crockerMethodsDigitalVideo1996`.

Initial and lifetime characteristics of long-lived bubbles are shown in **Table&NonBreakingSpace;1**. All bubbles departing the surface have an initial depth, $y$, close to the actual boiling surface, and a bimodal distribution in initial $x$, close to active nucleation sites. This information is used to determine surface and departure $y$ thresholds for alignment of bubble departures.


In [ ]:
tracks = (
    link(f=objects, search_range=SEARCH_RANGE, memory=MEMORY)
    .rename(columns={"x": "x_px", "y": "y_px"})
    .assign(
        frame_lifetime=(
            lambda df: df.groupby("particle", **GBC)["frame"].transform("count")
        )
    )
    .sort_values(["frame_lifetime", "particle", "frame"], ascending=[False, True, True])
    .assign(
        bubble=(lambda df: df.groupby("particle", **GBC).ngroup()),
        dy_px=lambda df: df.groupby("bubble", **GBC)[["y_px"]].diff().fillna(0),
        dx_px=lambda df: df.groupby("bubble", **GBC)[["x_px"]].diff().fillna(0),
        y=lambda df: df["y_px"] / PX_PER_M,
        x=lambda df: df["x_px"] / PX_PER_M,
        dy=lambda df: df["dy_px"] / PX_PER_M / frametime,
        dx=lambda df: df["dx_px"] / PX_PER_M / frametime,
        diameter=lambda df: df["diameter_px"] / PX_PER_M,
        radius_of_gyration=lambda df: df["radius_of_gyration_px"] / PX_PER_M,
        distance=lambda df: linalg.norm(df[["dx", "dy"]].abs(), axis=1),
        time=lambda df: video.sel(frame=df["frame"].values)["time"],
        lifetime=lambda df: df["frame_lifetime"] * frametime,
    )
    .drop(columns=["particle"])
)

with style_df(
    tracks.groupby("bubble", **GBC)
    .head(1)
    .set_index("bubble")
    .head(16)
    .pipe(
        transform_cols,
        cols=[
            Col("lifetime", "Lifetime", "s"),
            Col("time", r"$t_0$", "s"),
            Col("diameter", r"$d_{b0}$", **M_TO_MM),
            Col("y", r"$y_{b0}$", **M_TO_MM),
            Col("x", r"$x_{b0}$", **M_TO_MM),
        ],
    )
) as style:
    display_dataframe_with_math(style.background_gradient())

**Table&NonBreakingSpace;1**: Selected properties of long-lived bubbles
Bubbles are identified by a unique particle number. Their lifetime, the time of their first appearance, their initial diameter and elevation, and lifetime histograms of selected characteristics are shown.
